## Final Term Project Milestone 2
**April Meyer  
DSC540-T301 Data Preparation (2207-1)**

	1. Perform at least 5 data transformation and/or cleansing steps to your flat file data. For example:
        • Replace Headers
        • Format data into a more readable format
        • Identify outliers and bad data
        • Find duplicates
        • Fix casing or inconsistent values
        • Conduct Fuzzy Matching

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# reads excel file and stores as a dataframe
Books5 = pd.read_csv('Books.csv')
Books5.head()

,0001713353,A1C6M8LCIX4M6M,5.0,1123804800
0,0001713353,A1REUF3A1YCPHM,5.0,1112140800
1,0001713353,A1YRBRK2XM5D5,5.0,1081036800
2,0001713353,A1V8ZR5P78P4ZU,5.0,1077321600
3,0001713353,A2ZB06582NXCIV,5.0,1475452800
4,0001713353,ACPQVNRD3Z09X,5.0,1469750400


In [3]:
#1. Replace Headers
col_names = ['User_ID', 'ASIN_ISBN', 'Rating', 'DateTime']
Books5.columns = col_names
Books5.head()

,User_ID,ASIN_ISBN,Rating,DateTime
0,0001713353,A1REUF3A1YCPHM,5.0,1112140800
1,0001713353,A1YRBRK2XM5D5,5.0,1081036800
2,0001713353,A1V8ZR5P78P4ZU,5.0,1077321600
3,0001713353,A2ZB06582NXCIV,5.0,1475452800
4,0001713353,ACPQVNRD3Z09X,5.0,1469750400


In [4]:
#2. Converting the unix time to a datetime , 51,311,620
Books5['DateTime'] = pd.to_datetime(Books5['DateTime'],unit='s')
Books5.head()
Books5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51311620 entries, 0 to 51311619
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   User_ID    object        
 1   ASIN_ISBN  object        
 2   Rating     float64       
 3   DateTime   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.5+ GB


In [5]:
#3 Find a duplicate rows
duplicateRows_Books5 = Books5[Books5.duplicated()]
print(duplicateRows_Books5) #there is 142,050 duplicate rows.
#Removing duplicates
Books5.drop_duplicates(inplace=True) 
duplicateRows_Books5 = Books5[Books5.duplicated()]
print(duplicateRows_Books5) #there is 142,050 duplicate rows.

             User_ID       ASIN_ISBN  Rating   DateTime
1145      0001384198  A1DNVU9ZPA66AO     5.0 2011-11-07
1236      0001384198  A1TMAVN4CEM8U8     5.0 2007-09-04
1237      0001384198  A1TMAVN4CEM8U8     5.0 2007-09-04
1285      0001384198   AN4RKQM9AYVAD     5.0 2001-12-14
1473      0001381733  A3IVT3UA3ZERWQ     5.0 2016-02-18
...              ...             ...     ...        ...
51307107  B01GSGCTAI  A2MISP0MYPMHTS     5.0 2016-11-12
51308406  B01H0Q30YE  A39GAUE6TR0S2D     5.0 2018-08-31
51310232  B01HC1OF1Y  A35MIC8LAD097Z     5.0 2017-03-18
51310234  B01HC1OF1Y   A41H6RPH78ELT     5.0 2017-03-15
51310390  B01HCK7116  A140OBDLM9AHXX     5.0 2016-12-07

[142050 rows x 4 columns]
Empty DataFrame
Columns: [User_ID, ASIN_ISBN, Rating, DateTime]
Index: []


In [6]:
# 4. Looking for missing values
Books5.isnull().sum()

User_ID      0
ASIN_ISBN    0
Rating       0
DateTime     0
dtype: int64

In [7]:
#5. Transforming to get book ISB, rating, total ratings
Books5['TotalReviewsbyISBN'] = Books5.groupby('ASIN_ISBN')['ASIN_ISBN'].transform('count')
Books5['CountOfRatings'] = Books5.groupby(['ASIN_ISBN', 'Rating'])['ASIN_ISBN'].transform('count')
Books5.head()

,User_ID,ASIN_ISBN,Rating,DateTime,TotalReviewsbyISBN,CountOfRatings
0,0001713353,A1REUF3A1YCPHM,5.0,2005-03-30,50,35
1,0001713353,A1YRBRK2XM5D5,5.0,2004-04-04,2,2
2,0001713353,A1V8ZR5P78P4ZU,5.0,2004-02-21,1,1
3,0001713353,A2ZB06582NXCIV,5.0,2016-10-03,1,1
4,0001713353,ACPQVNRD3Z09X,5.0,2016-07-29,2,2


In [8]:
"""  NOTES BLOCK
SELECT state, count(name)
FROM df
GROUP BY state
ORDER BY state;

Python equivalent =
        n_by_state = df.groupby("state")["last_name"].count()
call .groupby() and pass the name of the column you want to group on, which is "state". 
use ["last_name"] to specify the columns on which you want to perform the actual aggregation.
"""
#Books5.drop(columns=['CountOfRatings'])

'  NOTES BLOCK\nSELECT state, count(name)\nFROM df\nGROUP BY state\nORDER BY state;\n\nPython equivalent =\n        n_by_state = df.groupby("state")["last_name"].count()\ncall .groupby() and pass the name of the column you want to group on, which is "state". \nuse ["last_name"] to specify the columns on which you want to perform the actual aggregation.\n'

In [9]:
#6. Creating a subset for ISBN, rating, count
Books5_subset = Books5[['ASIN_ISBN','Rating','TotalReviewsbyISBN', 'CountOfRatings']]
Books5_subset.head()

,ASIN_ISBN,Rating,TotalReviewsbyISBN,CountOfRatings
0,A1REUF3A1YCPHM,5.0,50,35
1,A1YRBRK2XM5D5,5.0,2,2
2,A1V8ZR5P78P4ZU,5.0,1,1
3,A2ZB06582NXCIV,5.0,1,1
4,ACPQVNRD3Z09X,5.0,2,2


In [13]:
# duplicateRows_Books5_subset = Books5_subset[Books5_subset.duplicated()]
# print(duplicateRows_Books5_subset) #there is 142,050 duplicate rows.
#Removing duplicates
Books5_subset.drop_duplicates(inplace=True) 
duplicateRows_Books5_subset = Books5_subset[Books5_subset.duplicated()]
print(duplicateRows_Books5_subset) #there is 142,050 duplicate rows.

C:\Users\swim_\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Empty DataFrame
Columns: [ASIN_ISBN, Rating, TotalReviewsbyISBN, CountOfRatings]
Index: []


In [14]:
Books5_subset.loc[Books5_subset['ASIN_ISBN'] == 'A1REUF3A1YCPHM']

,ASIN_ISBN,Rating,TotalReviewsbyISBN,CountOfRatings
0,A1REUF3A1YCPHM,5.0,50,35
8342064,A1REUF3A1YCPHM,3.0,50,3
11533096,A1REUF3A1YCPHM,4.0,50,12


In [15]:
Books5_subset.to_csv (r'C:\Users\swim_\DSC 540 Data Prep\amazon_reviews.csv', index = False, header=True)